# Main notebook

For quick and generic simulations.

## Imports

In [1]:
%matplotlib widget

import lppydsmc as ld
import plotting

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
from pathlib import Path

# you can choose the seed here
np.random.seed(1111)

## System choice

Four default systems can be initialized using the following cell.

In [2]:
system_type = 'thruster' # thruster, cylinder

# ---------------------- System --------------------
dz = 0.001
typical_lenght = 0.001 # typical size of the system (minimum distance between two walls for example)
                       # used for computing the mean free path later on (not used in the simulation)
                       # just useful to have an idea of it.

if system_type == 'tube':
    system, idx_out_walls, idx_in_wall = ld.systems.helper.system_rectangle(l_x = 0.01, l_y = 0.001)
    
elif system_type == 'square':
    system, idx_out_walls, idx_in_wall = ld.systems.helper.system_rectangle(l_x = 0.01, l_y = 0.01)

elif(system_type == 'cylinder'):
    system, idx_out_walls, idx_in_wall = ld.systems.helper.cylinder_system(res = 4, l_x = 0.003, l_y = 0.001, c_x = 0.0015 , c_y = 0.0005, r = 0.0001)

elif(system_type == 'thruster'):
    dp = 1e-3
    dict_thruster = {
        'w_in' : 5*dp,
        'l_in' : 3*dp,
        'w_1' : 3*dp,
        'l_1' : dp,
        'l_int' : dp,
        'w_2' : dp,
        'l_2' : 10*dp,
        'w_out' : 5*dp,
        'l_out' : dp,
        'offsets' : np.array([0,0]) 
    }

    system, idx_out_walls, idx_in_wall = ld.systems.helper.thruster_system(**dict_thruster)
    # idx_out_walls = [10, 9, 11] # not the input wall

elif(system_type == 'thruster_three_grids'):
    dp = 1e-3
    dict_thruster = {
        'w_in' : 5*dp,
        'l_in' : 3*dp,
        'w_1' : 3*dp,
        'l_1' : dp,
        'l_int' : dp,
        'w_2' : dp,
        'l_2' : 10*dp,
        'l_int_2' : dp,
        'w_3' : 3*dp,
        'l_3' : 1*dp,
        'w_out' : 5*dp,
        'l_out' : dp,
        'offsets' : np.array([0,0]) 
    }
    system, idx_out_walls, idx_in_wall = ld.systems.helper.thruster_three_grids_system(**dict_thruster)
    # idx_out_walls = [13, 14, 15] # not the input wall

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

## DSMC grid creation

In [3]:
# grid :
mean_number_per_cell = 500 # 200 is enough to have a "convergence", however, it is better to use more for better statistics (you could also average over more time steps in steady state)
max_number_per_cell = 10*mean_number_per_cell
# factor 10 is completely overshot (Note : a future version will have dynamic arrays instead of static one)

if system_type == 'tube':
    resolutions = np.array((10,1), dtype = int) # tube

elif system_type == 'square':
    resolutions = np.array((3,3), dtype = int) # tube

elif system_type == 'thruster':
    resolutions = np.array((16,5), dtype = int)

elif(system_type == 'cylinder'):
    resolutions = np.array((9,9), dtype = int)

elif system_type == 'thruster_three_grids':
    resolutions = np.array((18,5), dtype = int)
    
cells_number = np.prod(resolutions)
grid = ld.data_structures.Grid(cells_number, max_number_per_cell)

# --------- useful quantity for the simulation ------------ #
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]


## Particles

In [4]:
# ------------------ Particles Params ----------------- #

# particles density in the real system
density = 3.2e19 # m-3

part_type = 'I'
charge, mass, radius = 0, ld.utils.physics.get_mass_part(53, 53, 74), 2e-10

temperature = 300 # K 
v_mean = ld.utils.physics.maxwellian_mean_speed(temperature, mass)

size_array = max_number_per_cell*cells_number # max size for the array
container = ld.data_structures.Particle(part_type, charge, mass, radius, size_array)

# ------ useful quantities for the simulation (or for plotting) ----------- #
# ----------- based on params - should not be modified ---------------- #

# "mean number of particles in the simulated system"
n_simu = mean_number_per_cell * cells_number

# "mean number of particles in the real system"
n_real = volume_cell * density * cells_number 

# macro particules ratio = number of particles in the real system / number of macro part in the simulated system
mr = n_real/n_simu 

# density in the dsmc
density_dsmc = density/mr

# particle cross section (useful for particles collision)
cross_section = container.get_params()[3]  

# mean free path and time
mfp = ld.utils.physics.mean_free_path(cross_section, density)
mft = ld.utils.physics.mean_free_time(mfp, v_mean = v_mean) # min(typical_lenght,mfp)

## Initialization of particles

In [5]:
init_particles = False

if(init_particles):
    # you have to decice your strategy to initialize a 2D array of size Nx5
    # where N is the number of particle
    # and each particle is given [x, y, vx, vy, vz].
    init_size = mean_number_per_cell*np.prod(resolutions)
    extremal_values = system.get_extremal_values() 
    loc = 0
    vel_std = ld.utils.physics.gaussian(temperature, mass)
    x = np.random.uniform(low = extremal_values['min_x'], high = extremal_values['max_x'], size = init_size)
    y = np.random.uniform(low = extremal_values['min_y'], high = extremal_values['max_y'], size = init_size)
    vx = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    vy = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    vz = np.random.normal(loc=0.0, scale=vel_std, size = init_size)
    new = np.stack((x,y,vx,vy,vz), axis = 1) 

    container.add_multiple(new)

## Injection params 

In [6]:
inject_particles = True

if(inject_particles):
    in_wall = segments[idx_in_wall]
    in_vect = np.array([a[idx_in_wall,1], -a[idx_in_wall,0]]) # should be normalized
    in_vect = in_vect/np.linalg.norm(in_vect)
    
    # for the injection
    debit = ld.utils.physics.maxwellian_flux(density_dsmc, v_mean)*np.linalg.norm(in_wall[:2]-in_wall[2:])*dz
    vel_std = ld.utils.physics.gaussian(temperature, mass)


## Simulation parameters

In [7]:
# Simulation params
iterations = 1000
dt = 1e-6 # in sec.

# saving params 
saving_period = 10 # when do we save various data (see the simulation algo)
adding_period = 1 # when to we add to the dataframe that contains the particles position and velocity

# advection function - returns a 2D arrays containing the acceleration for each of the given particle
    # here arr is a N x 5 array. N particles, and for each one : x, y, vx, vy, vz is stored.
    # very simple for now - as there is no electric fields, nor any force in the system 
    # thus no acceleration
accelerations = np.zeros(shape = (size_array, 3))

def f(arr, t):
    return np.concatenate((arr[:,2:4], accelerations[:arr.shape[0]]), axis = 1)

# args is given to euler_explicit and then given to *f* (the advection function) in addition to arr and dt.
# in our case, it is not needed. However, we could imagine a system with an electric field computed at the setup phase, 
# and we would like to give it as an args.
args = []
scheme = ld.utils.schemes.euler_explicit # rk4; euler_explicit

## Summing-up and plotting

In [8]:
print(f'Initializing a system of type {system_type}, of shape {system_shape} with {np.prod(resolutions)} cells.')
print(f'The number of particles (type {part_type}) in the system is {container.get_current()}.')
print(f'Mean free path : {mfp} m ; Mean free time : {mft} m')
print(f'In steady state, {n_simu} can be expect in the simulated system, which represents {round(n_real,3)} in the real system. The ratio between the two is {round(mr,3)}.')
print(f'The simulation lasts {iterations} iterations, with a time step of {dt} s. Simulation duration : {dt*iterations} s')
# Note:  HDF5 uses a different format than csv, and the size on the disk is much different that what is expected. Check out : https://support.hdfgroup.org/HDF5/doc/H5.intro.html
# Here, you can at least multiply by 4 the size (considering we save much more than )
print(f'Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save {n_simu} particles each time is {iterations//adding_period*n_simu*(5*4)/1024**2} MB.') 
plotting.plt_tools.plot_system(container.get_array(), segments, radius, resolutions, system_shape, offsets);

Initializing a system of type thruster, of shape [0.016 0.005] with 80 cells.
The number of particles (type I) in the system is 0.
Mean free path : 0.04396075762485869 m ; Mean free time : 0.00019727117199744968 m
In steady state, 40000 can be expect in the simulated system, which represents 2560000000000.0 in the real system. The ratio between the two is 64000000.0.
The simulation lasts 1000 iterations, with a time step of 1e-06 s. Simulation duration : 0.001 s
Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save 40000 particles each time is 762.939453125 MB.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Saving directory and name

In [9]:
# which directory is used to save the data
# and under what name.
dir_path = Path('results/')
name = 'neutral_three_grids.h5' 

saver = ld.data.saver.Saver(dir_path, name)

# Simulation

The next cell takes care of the simulation. It algo gives you an idea of the evolution of the number of particles in the system and of its very general state.

At the end, the *saver* which saves the data is closed and you can then analyse your simulation using *analysis.ipynb*.

In [10]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# creating the positions in grids saver
# positions_in_grids_save = ld.data_structures.Container(size_array = container.get_max_size(), number_of_elements = 0, dtype=int)

# adding particle before the simulation - step 0
arr = container.get_array()
df = df.append(pd.DataFrame(data=arr, index=[0]*arr.shape[0], columns = ['x','y','vx','vy','vz']))

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

t = 0.0
for it in range(1,iterations+1): # tqdm
    n1 = container.get_current()
                   
    # ------------------------- INJECTING PARTICLES -------------------------
    new, remains = ld.injection.maxwellian(in_wall, in_vect, debit, vel_std, dt, remains)
    
    container.add_multiple(new)
        
    n2 = container.get_current()
    
    ############################
        # adding new particles to grid
    # positions = ld.data_structures.grid.default_hashing(ld.data_structures.grid.pos_in_grid(new[:,:2], resolutions, offsets, system_shape), res_y = resolutions[1])
    # parts_in_grid_format = ld.data_structures.grid.convert_to_grid_format(old = n1, new = n2)
    # grid.add_multiple(positions, parts_in_grid_format)
    
        # and to the positions in grid saver
    # positions_in_grids_save.add_multiple(positions)    
    ############################
    
    # ---------------------------- PHASE : ADVECTING --------------------
        # MOVING PARTICLES
    arr = container.get_array()
    ld.advection.advect(arr, f, dt, t, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    collisions_with_walls = 0
    while(np.count_nonzero(count) > 0): # np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp, cos_alpha = ld.advection.wall_collision.handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_, cos_alpha = ld.advection.wall_collision.make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count, cos_alpha) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
        
        # the first one that is received is the number of particles colliding with walls.
        if(c == 1):
            collisions_with_walls = np.count_nonzero(count) # np.sum(count, where = count == True)
    
    idxes_out = np.sort(np.concatenate(idxes_out))
    
    # ----------------------------- PHASE : UPDATING DATA BUCKETS ----------------------------- 
            # TODO : this is for one container (one species of particle)
        
    ############################
    # new_positions = ld.data_structures.grid.default_hashing(ld.data_structures.grid.pos_in_grid(arr[:,:2], resolutions, offsets, system_shape), res_y = resolutions[1])  
    # count = idxes_out.shape[0]-1
    # 
    # for idx in range(container.get_current()-1, -1, -1):
    #     old_pos = positions_in_grids_save.get(idx)
    #     new_pos = new_positions[idx]
    #     
    #     if(idxes_out.size > 0 and idx == idxes_out[count]):
    #         # particle is outside
    #         # we have to delete it
    #         current = container.get_current() # current changes with each delete !
    #         count -= 1

    #         # swapping it with the last one in the container
    #         container.delete(idx)
    #         positions_in_grids_save.delete(idx)
    #         # deleting the object in the grid
    #         grid.remove(old_pos, np.array([0,idx], dtype = int)) # since [idx container, idx] is a unique key, we check for equality in values of the array (not in references)

    #         # updating the swapped particle in the grid
    #         swapping_particle_pos = positions_in_grids_save.get(idx) # this supposes that positions_in_grids_save is up-to-date for the particle previously at index 'current-1' (and now at index 'idx'). 
    #         # this is why we are iterating from the end
    #         grid.update_index(swapping_particle_pos, idx_container = 0, old_index = current-1, new_index = idx)
    #     elif(np.array_equal(old_pos, new_pos)):
    #         pass
    #     else:
    #         # then the particle does not need to be deleted, just updated
    #         grid.update(o = np.array([0,idx]), old_pos = old_pos, new_pos = new_pos) # in theory it should not changed the value of the object (indeed, its position in the grid should not have changed !)
    #         # and update the new positions in grid in the saver
    #         positions_in_grids_save.update(idx, new_pos)
            
    # arr = container.get_array()
    ############################
    
    ############################
    container.delete_multiple(idxes_out)
    arr = container.get_array()
    new_positions = ld.data_structures.grid.default_hashing(ld.data_structures.grid.pos_in_grid(arr[:,:2], resolutions, offsets, system_shape), res_y = resolutions[1])  
    grid.reset()
    parts_in_grid_format = ld.data_structures.grid.convert_to_grid_format(new = new_positions.shape[0])
    grid.add_multiple(new_positions, parts_in_grid_format)
    ############################

    # ----------------------------- PHASE : DSMC COLLISIONS ----------------------------- 
        # TODO: make parallel (1st : note criticals functions in C++)
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # TODO: may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = ld.collision.handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    
    t += dt
    
    # ----------------------------- PLOTTING AND SAVING (OPTIONAL) ----------------------------- 
    if(it%adding_period == 0 or it == iterations-1):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
        
    if(it%saving_period == 0 or it == iterations-1): # saving if last iteration too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages, # evolution of the average number of particle per cell
                        'collisions_with_walls' : collisions_with_walls, # number of collisions with walls - evolution

                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz'])
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2-n1, idxes_out.shape[0], c))
saver.close()

|    it    |   INIT   |  INJECT  |   DEL    |    TRY   |
--------------------------------------------------------
|    10    |   1253   |   139    |    0     |    3     |
|    20    |   2633   |   139    |    3     |    3     |
|    30    |   3901   |   139    |    28    |    3     |
|    40    |   4976   |   140    |    37    |    3     |
|    50    |   5931   |   139    |    65    |    3     |
|    60    |   6688   |   139    |    68    |    3     |
|    70    |   7309   |   139    |    87    |    3     |
|    80    |   7852   |   140    |    89    |    3     |
|    90    |   8327   |   139    |   112    |    3     |
|   100    |   8711   |   139    |   109    |    3     |
|   110    |   9050   |   139    |   116    |    3     |
|   120    |   9260   |   139    |    96    |    3     |
|   130    |   9544   |   140    |   112    |    3     |
|   140    |   9753   |   139    |   114    |    3     |
|   150    |   9980   |   139    |   137    |    3     |
|   160    |  10105   |   139  

KeyboardInterrupt: 